In [1]:
import pandas as pd
import numpy as np

season_data = pd.read_csv("pbp_2017.csv", low_memory=False)


In [2]:
#Relevant Columns

cols = ['Date',
'GameID',
'Drive',
'qtr',
'down',
'TimeSecs',
'PlayTimeDiff',
'SideofField',
'yrdline100',
'ydstogo',
'ydsnet',
'GoalToGo',
'FirstDown',
'posteam',
'DefensiveTeam',
'PlayAttempted',
'Yards.Gained',
'sp',
'Touchdown',
'ExPointResult',
'TwoPointConv',
'DefTwoPoint',
'Safety',
'PlayType',
'Passer',
'Passer_ID',
'PassAttempt',
'PassOutcome',
'PassLength',
'AirYards',
'YardsAfterCatch',
'QBHit',
'PassLocation',
'InterceptionThrown',
'Rusher',
'Rusher_ID',
'RushAttempt',
'RunLocation',
'RunGap',
'Receiver',
'Receiver_ID',
'Reception',
'Fumble',
'RecFumbTeam',
'Sack',
'PosTeamScore',
'DefTeamScore',
'ScoreDiff',
'AbsScoreDiff',
'HomeTeam',
'AwayTeam',
'Season']

season_data = season_data[cols]

In [3]:
#TOTAL Season Passing Points

def passing_points(passer):
    #Establishing Dataframes
    passer_df = season_data[season_data['Passer']==passer]
    passing_df = passer_df[passer_df['PassOutcome']=='Complete']
    
    #Total Passing Yards
    pd.to_numeric(passing_df['AirYards'])
    pd.to_numeric(passing_df['YardsAfterCatch'])
    
    total_pass_yards = passing_df['AirYards'].sum() + passing_df['YardsAfterCatch'].sum()
    total_pass_points = .04 * total_pass_yards
    
    #Interceptions
    pd.to_numeric(passer_df['InterceptionThrown'])
    total_interceptions = passer_df['InterceptionThrown'].sum()
    total_interception_points = (-1) * total_interceptions
    
    #Passing Touchdowns
    pd.to_numeric(passing_df['Touchdown'])
    total_pass_td = passing_df['Touchdown'].sum()
    total_pass_td_points = 4 * total_pass_td

    #300+ Yard Games
    game_list = passing_df['GameID'].unique().tolist()
    three_hund_plus = 0
    
    for game in game_list:
        game_df = passing_df[passing_df['GameID'] == game]
        game_yards = game_df['AirYards'].sum() + game_df['YardsAfterCatch'].sum()
        if game_yards >= 300:
            three_hund_plus += 1
    
    total_three_hund_plus = 3 * three_hund_plus 
    
    #Total DK Passing Points
    total_dk_passing_points = total_pass_points + total_interception_points + total_pass_td_points + total_three_hund_plus
    return total_dk_passing_points


In [4]:
#TOTAL Season Rushing Points

def rushing_points(rusher):
    #Establishing Dataframe
    rusher_df = season_data[season_data['Rusher']==rusher]
    
    #Rushing Touchdowns
    pd.to_numeric(rusher_df['Touchdown'])
    total_rushing_touchdowns = rusher_df['Touchdown'].sum()
    total_rushing_td_points = 6 * total_rushing_touchdowns
    
    #Rushing Yards
    pd.to_numeric(rusher_df['Yards.Gained'])
    total_rushing_yards = rusher_df['Yards.Gained'].sum()
    total_rushing_yd_points = 0.1 * total_rushing_yards
    
    #100+ Yard Games
    game_list = rusher_df['GameID'].unique().tolist()
    one_hund_plus = 0
    
    for game in game_list:
        game_df = rusher_df[rusher_df['GameID'] == game]
        game_yards = game_df['Yards.Gained'].sum()
        if game_yards >= 100:
            one_hund_plus += 1
    
    total_one_hund_plus = 3 * one_hund_plus 
    
    #Fumbles
    pd.to_numeric(rusher_df['Fumble'])
    total_fumbles = rusher_df['Fumble'].sum()
    total_fumbles_points = (-1) * total_fumbles
    
    
    #Total DK Rushing Points
    total_dk_rushing_points = total_rushing_td_points + total_rushing_yd_points + total_one_hund_plus + total_fumbles_points
    
    return(total_dk_rushing_points)


In [5]:
#TOTAL Season Receiving Points

def receiver_points(receiver):
    #Establishing Dataframe
    receiver_df = season_data[season_data['Receiver']==receiver]
    reception_df = receiver_df[receiver_df['PassOutcome']=='Complete']
    
    #Receiving Touchdowns
    pd.to_numeric(receiver_df['Touchdown'])
    total_receiving_touchdowns = receiver_df['Touchdown'].sum()
    total_receiving_td_points = 6 * total_receiving_touchdowns
    
    #Receiving Yards
    total_receiving_yards = reception_df['AirYards'].sum() + reception_df['YardsAfterCatch'].sum()
    total_receiving_yd_points = 0.1 * total_receiving_yards
    
    #100+ Yard Games
    game_list = reception_df['GameID'].unique().tolist()
    one_hund_plus = 0
    
    for game in game_list:
        game_df = reception_df[reception_df['GameID'] == game]
        game_yards = game_df['AirYards'].sum() + game_df['YardsAfterCatch'].sum()
        if game_yards >= 100:
            one_hund_plus += 1
    
    total_one_hund_plus = 3 * one_hund_plus 
    
    #PPR
    receptions = len(reception_df)
    
    #Fumbles
    pd.to_numeric(reception_df['Fumble'])
    total_fumbles = reception_df['Fumble'].sum()
    total_fumbles_points = (-1) * total_fumbles
    
    
    #Total DK Receiver Points
    total_dk_receiving_points = total_receiving_td_points + total_receiving_yd_points + total_one_hund_plus + total_fumbles_points + receptions  
    
    return(total_dk_receiving_points)


In [6]:
def total_offensive_points(player):
    passing_pts = passing_points(player)
    rushing_pts = rushing_points(player)
    receiving_pts = receiver_points(player)
    
    total_offensive_points = passing_pts + rushing_pts + receiving_pts
    
    return(total_offensive_points)

In [7]:
#Building DF for top 30ish quarterbacks
quarterbacks = season_data['Passer'].dropna().unique().tolist()
#Players that are either not full-time quarterbacks or are obvious errors (T.Hill and T.Taylor)
non_quarterbacks = ['C.Kupp','T.Cohen','M.Lee', 'J.Landry', 'L.Fitzgerald', 'S.Vereen', 'T.Kelce', 'G.Tate', 'T.Taylor', 'T.Hill', 'M.Sanu', 'R.Cobb', 'E.Decker', 'D.Stanton', 'M.Moore', 'M.Glennon', 'T.Yates', 'K.Hogan', 'N.Foles', 'S.Bradford', 'B.Petty', 'D.Fales', 'P.Lynch', 'E.Manuel', 'P.Mahomes', 'T.McEvoy', 'R.Mallett', 'N.Peterman', 'G.Smith', 'L.Jones', 'T.Bray', 'M.Cassel', 'S.Mannion', 'N.Sudfeld', 'J.Webb', 'C.Kessler', 'S.Tolzien', 'M.Gray', 'A.McCarron', 'C.Jones', 'C.Rush', 'R.Golden', 'S.Koch', 'J.Hekker', 'L.Edwards', 'B.Nortman', 'D.Anderson', 'T.Heinicke', 'J.Callahan', 'K.Clemens', 'R.Quigley', 'C.Henne', 'M.Palardy', 'M.Hull', 'J.Rudock', 'J.Ryan', 'T.Bridgewater', 'M.Haack']

for non_qb in non_quarterbacks:
    try:
        quarterbacks.remove(non_qb)
    except:
        pass

qb_ranks = {}
for player in quarterbacks:
    points = total_offensive_points(player)
    qb_ranks[player] = points

#Produces dataframe of just quarterbacks and their overall season points
qb_ranks_df = pd.DataFrame.from_dict(qb_ranks, orient='index', columns=['Total Points'])
pd.to_numeric(qb_ranks_df['Total Points'])

#Prettying up the QB Dataframe
qb_ranks_df = qb_ranks_df['Total Points'].sort_values(ascending=False)
quarterbacks = pd.DataFrame(data=qb_ranks_df)
quarterbacks['Player'] = quarterbacks.index
quarterbacks.reset_index(inplace=True, level=None)
quarterbacks = quarterbacks.drop(['index'], axis=1)
quarterbacks = quarterbacks[['Player', 'Total Points']]

In [8]:
#Building DF for top 30-50 running backs
rushers = season_data['Rusher'].dropna().unique().tolist()

#Trying to clean out any quarterbacks from running backs list
for qb in quarterbacks['Player']:
    if qb in rushers:
        try:
            rushers.remove(qb)
        except:
            pass
#Players that rushed the ball at least once but are not running backs - at least pulled from top 30 point producers
non_rushers = ['D.Thompson', 'M.Bryant', 'R.Matthews', 'D.Jackson', 'M.Lee','D.Njoku', 'T.Smith', 'K.Golladay', 'C.Patterson', 'O.Beckham', 'A.Humphries', 'T.Benjamin', 'T.Lockett', 'A.Wilson', 'J.Doctson', 'T.Williams', 'W.Fuller', 'C.Hogan', 'D.Thomas', 'T.Taylor', 'K.Allen', 'Ju.Jones', 'T.Hill', 'T.Kelce', 'J.Nelson', 'G.Tate', 'D.Baldwin', 'B.Cooks', 'M.Jones', 'R.Anderson', 'S.Diggs', 'N.Agholor', 'J.Landry', 'A.Thielen', 'D.Bryant', 'M.Sanu', 'D.Walker', 'E.Engram', 'J.Crowder', 'M.Goodwin', 'A.Cooper', 'R.Woods', 'T.Ginn', 'R.Cobb', 'S.Shepard', 'M.Wallace']        

for player in non_rushers:
    if player in rushers:
        try:
            rushers.remove(player)
        except:
            pass

rb_ranks = {}
for player in rushers:
    points = total_offensive_points(player)
    rb_ranks[player] = points

#Produces dataframe of just quarterbacks and their overall season points
rb_ranks_df = pd.DataFrame.from_dict(rb_ranks, orient='index', columns=['Total Points'])
pd.to_numeric(rb_ranks_df['Total Points'])

#Prettying up Running back Data Frame
rb_ranks_df = rb_ranks_df['Total Points'].sort_values(ascending=False)
running_backs = pd.DataFrame(data=rb_ranks_df)
running_backs['Player'] = running_backs.index
running_backs.reset_index(inplace=True, level=None)
running_backs = running_backs.drop(['index'], axis=1)
running_backs = running_backs[['Player', 'Total Points']]



In [9]:
#Building average weekly points for each player / DST
def avg_points(total_points):
    average = total_points / 16
    return average

running_backs['Avg. Points'] = running_backs['Total Points'].apply(avg_points)



In [10]:
#Season Broken into weeks
week_one = ['2017-09-07','2017-09-10','2017-09-11']
week_two = ['2017-09-14', '2017-09-17','2017-09-18' ]
week_three = ['2017-09-21', '2017-09-24', '2017-09-25']
week_four = ['2017-09-28', '2017-10-01', '2017-10-02']
week_five = ['2017-10-05', '2017-10-08', '2017-10-09']
week_six = ['2017-10-12', '2017-10-15', '2017-10-16']
week_seven = ['2017-10-19', '2017-10-22', '2017-10-23']
week_eight = ['2017-10-26', '2017-10-29', '2017-10-30']
week_nine = ['2017-11-02', '2017-11-05', '2017-11-06' ]
week_ten = ['2017-11-09', '2017-11-12', '2017-11-13']
week_eleven = ['2017-11-16', '2017-11-19', '2017-11-20']
week_twelve = ['2017-11-23', '2017-11-26', '2017-11-27']
week_thirteen = ['2017-11-30', '2017-12-03', '2017-12-04']
week_fourteen = ['2017-12-07', '2017-12-10', '2017-12-11']
week_fifteen = ['2017-12-14', '2017-12-16', '2017-12-17']
week_sixteen = ['2017-12-18', '2017-12-23', '2017-12-24', '2017-12-25']
week_seventeen = ['2017-12-31']

all_the_weeks = [week_one, week_two, week_three, week_four, week_five, week_six, week_seven, week_eight, week_nine, week_ten, week_eleven, week_twelve, week_thirteen, week_fourteen, week_fifteen, week_sixteen, week_seventeen]
all_the_weeks_past_one = [week_two, week_three, week_four, week_five, week_six, week_seven, week_eight, week_nine, week_ten, week_eleven, week_twelve, week_thirteen, week_fourteen, week_fifteen, week_sixteen, week_seventeen]

def weekly_offense_points(player, week):

    current_week_df = pd.DataFrame()
    for game_day in week:
        game_day_df = season_data[season_data['Date']==game_day]
        current_week_df = pd.concat([current_week_df, game_day_df], ignore_index=True)
    
    #Rushing Stuff
    rusher_df = current_week_df[current_week_df['Rusher']==player]
    
    opponent = str(rusher_df['DefensiveTeam'].unique())
    opponent = opponent.replace("'","")
    opponent = opponent.replace("[", "")
    opponent = opponent.replace("]","")
    
    #Home and Away Teams
    away_team = str(rusher_df['AwayTeam'].unique())
    away_team = away_team.replace("'","")
    away_team = away_team.replace("[", "")
    away_team = away_team.replace("]","")
    home_team = str(rusher_df['HomeTeam'].unique())
    home_team = home_team.replace("'","")
    home_team = home_team.replace("[", "")
    home_team = home_team.replace("]","")
    
    #Rushing Touchdowns
    pd.to_numeric(rusher_df['Touchdown'])
    total_rushing_touchdowns = rusher_df['Touchdown'].sum()
    total_rushing_td_points = 6 * total_rushing_touchdowns
    
    #Rushing Yards
    pd.to_numeric(rusher_df['Yards.Gained'])
    total_rushing_yards = rusher_df['Yards.Gained'].sum()
    total_rushing_yd_points = 0.1 * total_rushing_yards
    
    #Fumbles
    pd.to_numeric(rusher_df['Fumble'])
    total_fumbles = rusher_df['Fumble'].sum()
    total_fumbles_points = (-1) * total_fumbles
    
    #Catch Dat Ball
    receiver_df = current_week_df[current_week_df['Receiver']==player]
    reception_df = receiver_df[receiver_df['PassOutcome']=='Complete']
    
    #Receiving Touchdowns
    pd.to_numeric(receiver_df['Touchdown'])
    total_receiving_touchdowns = receiver_df['Touchdown'].sum()
    total_receiving_td_points = 6 * total_receiving_touchdowns
    
    #Receiving Yards
    total_receiving_yards = reception_df['AirYards'].sum() + reception_df['YardsAfterCatch'].sum()
    total_receiving_yd_points = 0.1 * total_receiving_yards
    
    #Targets
    total_targets = len(receiver_df)
    
    #PPR
    receptions = len(reception_df)
    
    #Fumbles after reception
    pd.to_numeric(reception_df['Fumble'])
    total_fumbles = reception_df['Fumble'].sum()
    total_fumbles_points_rec = (-1) * total_fumbles

    #Sum that shit up
    total_week_points = total_rushing_td_points + total_rushing_yd_points + total_fumbles_points + total_receiving_td_points + total_receiving_yd_points + total_fumbles_points_rec + receptions
    
    #Motherfucking conditionals
    
    if total_rushing_yards >= 100:
        total_week_points+= 3
    
    if total_receiving_yards >= 100:
        total_week_points += 3
        
    
    weekly_offense_stats = [total_week_points, total_rushing_yards, total_rushing_touchdowns, total_targets, receptions, total_receiving_yards, total_receiving_touchdowns, opponent, home_team, away_team]
    
    return(weekly_offense_stats)


    


In [11]:
#season averages columns

def rb_season_averages(player):
    rusher_df = season_data[season_data['Rusher']==player]
    receiver_df = season_data[season_data['Receiver']==player]
    reception_df = receiver_df[receiver_df['PassOutcome']=='Complete']
    
    rb_averages = []
    
    games_played = len(rusher_df['Date'].unique())
    
    #Total Receiving Yards
    pd.to_numeric(reception_df['AirYards'])
    pd.to_numeric(reception_df['YardsAfterCatch'])
    
    total_receiving_yards = reception_df['AirYards'].sum() + reception_df['YardsAfterCatch'].sum()
    average_receiving_yards = total_receiving_yards / float(games_played)
    
    #Rushing Opportunities
    total_rushing_opps = len(rusher_df)
    average_rushing_opps = total_rushing_opps / float(games_played)
    
    #Red Zone Opportunities
    pd.to_numeric(rusher_df['yrdline100'])
    pd.to_numeric(receiver_df['yrdline100'])
    red_zone_rush_df = rusher_df[rusher_df['yrdline100'] >= 80]
    red_zone_rec_df = receiver_df[receiver_df['yrdline100'] >= 80]
    
    red_zone_opps = len(red_zone_rush_df) + len(red_zone_rec_df)
    
    #Avg Red Zone Opps
    
    avg_red_zone_opps = red_zone_opps / float(games_played)
    
    #Targets
    average_targets = len(receiver_df) / float(games_played)
    
    #Receptions
    average_receptions = len(reception_df) / float(games_played)
    
    #Receiving Touchdowns
    pd.to_numeric(reception_df['Touchdown'])
    total_receiving_td = reception_df['Touchdown'].sum()
    average_receiving_tds = total_receiving_td / float(games_played)
    
    #Rushing Touchdowns
    pd.to_numeric(rusher_df['Touchdown'])
    total_rushing_touchdowns = rusher_df['Touchdown'].sum()
    average_rush_tds = total_rushing_touchdowns / float(games_played)
    
    #Rushing Yards
    pd.to_numeric(rusher_df['Yards.Gained'])
    total_rushing_yards = rusher_df['Yards.Gained'].sum()
    average_rush_yards = total_rushing_yards / float(games_played)
    
    rb_averages = [average_rush_yards, average_rush_tds, average_targets, average_receptions, average_receiving_yards, average_receiving_tds, total_rushing_opps, average_rushing_opps, red_zone_opps, avg_red_zone_opps, games_played]

    return(rb_averages)

running_backs_list = running_backs['Player'].tolist()

def average_columns(val):
    average_calc = []
    for qb in running_backs_list:
        average_calc.append(rb_season_averages(qb)[val])
        
    return(average_calc)

running_backs['Avg Rush Yds'] = average_columns(0)
running_backs['Avg Rush Tds'] = average_columns(1)
running_backs['Avg Targets'] = average_columns(2)
running_backs['Avg Receptions'] = average_columns(3)
running_backs['Avg Receiving Yards'] = average_columns(4)
running_backs['Avg Receiving Tds'] = average_columns(5)
running_backs['Total Rushing Opps'] = average_columns(6)
running_backs['Avg Rushing Opps'] = average_columns(7)
running_backs['Red Zone Opps Rush/Pass'] = average_columns(8)
running_backs['Average Red Zone Opps'] = average_columns(9)
running_backs['Games Played'] = average_columns(10)




In [ ]:
#running backs weekly data

def column_creator(val):
    current_week_pts = []
    for rb in running_backs_list:
        current_week_pts.append(weekly_offense_points(rb,week_one)[val])

    return(current_week_pts)

running_backs['Week 1 pts'] = column_creator(0)
running_backs['Week 1 Rush Yds'] = column_creator(1)
running_backs['Week 1 Rush TDs'] = column_creator(2)
running_backs['Week 1 Targets'] = column_creator(3)
running_backs['Week 1 Receptions'] = column_creator(4)
running_backs['Week 1 Receiving Yds'] = column_creator(5)
running_backs['Week 1 Receiving Tds'] = column_creator(6)
running_backs['Week 1 Opponent'] = column_creator(7)
running_backs['Week 1 Home Team'] = column_creator(8)
running_backs['Week 1 Away Team'] = column_creator(9)

for i in range(2,17):
    week = all_the_weeks_past_one[i-2]
    def column_creator_v2(val):
        current_week_pts = []
        for rb in running_backs_list:
            current_week_pts.append(weekly_offense_points(rb, week)[val])
        
        return(current_week_pts)
        
    running_backs['Week '+ str(i) + ' pts'] = column_creator_v2(0)
    running_backs['Week '+ str(i) + ' Rush Yds'] = column_creator_v2(1)
    running_backs['Week '+ str(i) + ' Rush TDs'] = column_creator_v2(2)
    running_backs['Week '+ str(i) + ' Targets'] = column_creator_v2(3)
    running_backs['Week '+ str(i) + ' Receptions'] = column_creator_v2(4)
    running_backs['Week '+ str(i) + ' Receiving Yds'] = column_creator_v2(5)
    running_backs['Week '+ str(i) + ' Receiving Tds'] = column_creator_v2(6)
    running_backs['Week '+ str(i) + ' Opponent'] = column_creator_v2(7)
    running_backs['Week '+ str(i) + ' Home Team'] = column_creator_v2(8)
    running_backs['Week '+ str(i) + ' Away Team'] = column_creator_v2(9)
        



In [ ]:
running_backs.to_csv('running_backs_2017_home_away.csv', sep=',')